# Atividade Complementar - AP3 - Implementação da Máquina de Turing em Python

> **Objetivo**: Implementar uma máquina de Turing decisora que reconheça à linguagem `x#y`, em que x e y são números denotados na linguagem unária com símbolos `I`. A máquina, além de reconhecer as palavras da linguagem especificada, deve imprimir, ao final da entrada, o resultado de `x mod y` e a palavra `ACEITA`.  Quando não for possível, deve escrever apenas a palavra `REJEITA`.

## Time
Orientadora: Elloá Guedes - [@elloa](https://github.com/elloa)
* Gabriel Dos Santos Lima - [@gabrielSantosLima](https://github.com/gabrielSantosLima)
* Cássia Jamilly Venâncio - [@Cassialic](https://github.com/Cassialic)



## Etapa 1: Máquina de Turing no JFLAP

> Alguns nomes de estados podem estar nomeados fora de ordem (numérica), mas o código na etapa seguinte segue o exato nome de estados mostrados no diagrama.

A imagem a seguir mostra o diagrama para a máquina de Turing que resolve o problema apresentado. Com base neste diagrama, realizaremos a implementação em Python nas etapas posteriores.

<center>

![](https://github.com/uea-geral/ftc-turing-machine/blob/main/schema.jpg?raw=true)

</center>

## Etapa 2: Máquina de Turing em Python

In [187]:
class Logger:
  def __init__(self):
    self.logs = []

  def add(self, log: str):
    self.logs.append(log)

  def show(self):
    print("\n".join(self.logs))

In [188]:
EMPTY = '⊔'

class TuringMachine:
  def __init__(self):
    self.tape = []
    self.head = 0
    self.logger = Logger()

  def __reset_configurations(self):
    self.tape = []
    self.head = 0

  def __get_tape(self):
    # for better visualization, we are removing EMPTY symbol of tape
    sanitized_tape = [c for c in self.tape if c != EMPTY]
    return "".join(sanitized_tape)

  def __current_is(self, symbol: str):
    return self.tape[self.head] == symbol

  def __register_tape_configuration(self):
    merged_tape = self.__get_tape()
    if self.head == 0:
      self.logger.add(f">{merged_tape}")
    else:
      self.logger.add(f"{merged_tape[:self.head-1]}>{merged_tape[self.head:]}")

  def __move_left(self):
    self.head -= 1
    self.__register_tape_configuration()

  def __move_right(self):
    self.head += 1
    self.__register_tape_configuration()

  def __write(self, symbol: str):
    self.tape[self.head] = symbol

  # =================== CHECKING VALIDATION ===================
  def q0(self):
    if self.__current_is('I'):
      self.__move_right()
      self.q34()
    elif self.__current_is('#'):
      self.__move_right()
      self.q32()

  def q32(self):
    if self.__current_is(EMPTY):
      self.__write(' ')
      self.__move_right()
      self.q_reject()
    elif self.__current_is('#') or self.__current_is('I'):
      self.__move_right()
      self.q32()

  def q34(self):
    if self.__current_is('I'):
      self.__move_right()
      self.q34()
    elif self.__current_is('#'):
      self.__move_right()
      self.q35()
    elif self.__current_is(EMPTY):
      self.__write(' ')
      self.__move_right()
      self.q_reject()

  def q35(self):
    if self.__current_is('I'):
      self.__move_right()
      self.q1()
    elif self.__current_is(EMPTY):
      self.__write(' ')
      self.__move_right()
      self.q_reject()

  def q1(self):
    if self.__current_is('I'):
      self.__move_right()
      self.q1()
    elif self.__current_is('#'):
      self.__move_right()
      self.q2()
    elif self.__current_is(EMPTY):
      self.__move_left()
      self.q3()

  def q2(self):
    if self.__current_is('#') or self.__current_is('I'):
      self.__move_right()
      self.q2()
    elif self.__current_is(EMPTY):
      self.__write(' ')
      self.__move_right()
      self.q_reject()

  # =================== CALCULATING MOD ===================
  def q3(self):
    if self.__current_is('I'):
      self.__move_left()
      self.q3()
    elif self.__current_is('#'):
      self.__move_left()
      self.q4()

  def q4(self):
    if self.__current_is('X'):
      self.__move_left()
      self.q4()
    elif self.__current_is('I'):
      self.__write('X')
      self.__move_right()
      self.q5()
    elif self.__current_is(EMPTY):
      self.__move_right()
      self.q10()

  def q5(self):
    if self.__current_is('X'):
      self.__move_right()
      self.q5()
    elif self.__current_is('#'):
      self.__move_right()
      self.q6()

  def q6(self):
    if self.__current_is('X'):
      self.__move_right()
      self.q6()
    elif self.__current_is('I'):
      self.__write('X')
      self.__move_left()
      self.q7()
    elif self.__current_is(EMPTY):
      self.__move_left()
      self.q8()

  def q7(self):
    if self.__current_is('X'):
      self.__move_left()
      self.q7()
    elif self.__current_is('#'):
      self.__move_left()
      self.q4()

  def q8(self):
    if self.__current_is('X'):
      self.__write('I')
      self.__move_left()
      self.q8()
    elif self.__current_is('#'):
      self.__move_right()
      self.q9()

  def q9(self):
    if self.__current_is('I'):
      self.__write('X')
      self.__move_left()
      self.q16()

  def q16(self):
    if self.__current_is('#'):
      self.__move_left()
      self.q4()


  # =================== MOVING MOD TO THE END OF THE TAPE ===================
  def q10(self):
    if self.__current_is('X'):
      self.__write('I')
      self.__move_right()
      self.q10()
    elif self.__current_is('#'):
      self.__move_right()
      self.q33()

  def q11(self):
    if self.__current_is('I') or self.__current_is('X'):
      self.__move_right()
      self.q11()
    elif self.__current_is(EMPTY):
      self.__write('=')
      self.__move_left()
      self.q12()

  def q12(self):
    if self.__current_is('I'):
      self.__move_left()
      self.q12()
    elif self.__current_is('X'):
      self.__write('I')
      self.__move_right()
      self.q13()
    elif self.__current_is('#'):
      self.__move_right()
      self.q15()

  def q13(self):
    if self.__current_is('=') or self.__current_is('X') or self.__current_is('I'):
      self.__move_right()
      self.q13()
    elif self.__current_is(EMPTY):
      self.__write('X')
      self.__move_left()
      self.q14()

  def q14(self):
    if self.__current_is('X'):
      self.__move_left()
      self.q14()
    elif self.__current_is('='):
      self.__move_left()
      self.q12()

  def q15(self):
    if self.__current_is('=') or self.__current_is('I'):
      self.__move_right()
      self.q15()
    elif self.__current_is('X'):
      self.__write('I')
      self.__move_right()
      self.q15()
    elif self.__current_is(EMPTY):
      self.__write(' ')
      self.__move_right()
      self.q_accept()

  def q33(self):
    if self.__current_is('X'):
      self.__move_right()
      self.q33()
    elif self.__current_is('I'):
      self.__move_right()
      self.q11()
    elif self.__current_is(EMPTY):
      self.__write('=')
      self.__move_left()
      self.q37()

  def q36(self):
    if self.__current_is(EMPTY):
      self.__write(' ')
      self.__move_right()
      self.q_accept()

  def q37(self):
    if self.__current_is('X'):
      self.__write('I')
      self.__move_left()
      self.q37()
    elif self.__current_is('#'):
      self.__move_right()
      self.q38()

  def q38(self):
    if self.__current_is('I'):
      self.__move_right()
      self.q38()
    elif self.__current_is('='):
      self.__move_right()
      self.q36()

  # =================== ACCEPTING WORD ===================
  def q_accept(self):
    self.__write('A')
    self.__move_right()
    self.__write('C')
    self.__move_right()
    self.__write('E')
    self.__move_right()
    self.__write('I')
    self.__move_right()
    self.__write('T')
    self.__move_right()
    self.__write('A')
    self.__move_right()

  # =================== REJECTING WORD ===================
  def q_reject(self):
    self.__write('R')
    self.__move_right()
    self.__write('E')
    self.__move_right()
    self.__write('J')
    self.__move_right()
    self.__write('E')
    self.__move_right()
    self.__write('I')
    self.__move_right()
    self.__write('T')
    self.__move_right()
    self.__write('A')
    self.__move_right()


  def compute(self, w: str, verbose=False):
    self.__reset_configurations()

    # initialing tape...
    self.tape.append(EMPTY)
    for character in w:
      self.tape.append(character)
    self.tape.append(EMPTY)
    self.tape.append(EMPTY)
    self.tape.append(EMPTY)
    self.tape.append(EMPTY)
    self.tape.append(EMPTY)
    self.tape.append(EMPTY)
    self.tape.append(EMPTY)
    self.tape.append(EMPTY)
    self.tape.append(EMPTY)
    self.tape.append(EMPTY)
    self.tape.append(EMPTY)
    self.tape.append(EMPTY) # adding multiple empty values
    self.logger.add(f"[INIT] {self.__get_tape()}")

    # computing...
    self.__move_right()
    self.q0()

    if verbose:
      self.logger.show()

    return self.__get_tape()

## Etapa 3: Testes

> **IMPORTANTE**: Para visualizar os _logs_ dos movimentos do cabeçote, passar para a função `TuringMachine.computate()` o parâmetro **verbose=True**. Para melhor visualização do resultado final, optamos por deixar com valor falso por padrão.

A seguir testaremos várias palavras distintas, dentre as quais as que foram apresentadas no enunciado deste projeto:

<center>

|Entrada|Saída Esperada|
|-------|--------------|
|IIIIIII#III|IIIIIII#III=I ACEITA|
|I#I#I#|I#I#I# REJEITA|
|IIIIIII#IIII|IIIIIII#IIII=III ACEITA|
|IIIIIIIII#IIIII|IIIIIIIII#IIIII=IIII ACEITA|
|IIIIIIIIIIIIII|IIIIIIIIIIIIII REJEITA|

</center>


Antes de testá-los, criaremos uma função de utilidade para execução dos testes:

In [189]:
def expect(output, expected):
  if output != expected:
    raise Exception(f"[TEST FAILED] \nExpected: {expected}\nReceived: {output}")
  print(f"Output: {output}")
  print("Success!")

In [190]:
tm = TuringMachine()

In [191]:
expect(tm.compute("IIIIIII#III"), "IIIIIII#III=I ACEITA")

Output: IIIIIII#III=I ACEITA
Success!


In [192]:
expect(tm.compute("I#I#I#"), "I#I#I# REJEITA")

Output: I#I#I# REJEITA
Success!


In [193]:
expect(tm.compute("IIIIIII#IIII"), "IIIIIII#IIII=III ACEITA")

Output: IIIIIII#IIII=III ACEITA
Success!


In [194]:
expect(tm.compute("IIIIIIIII#IIIII"), "IIIIIIIII#IIIII=IIII ACEITA")

Output: IIIIIIIII#IIIII=IIII ACEITA
Success!


In [195]:
expect(tm.compute("IIIIIIIIIIIIII"), "IIIIIIIIIIIIII REJEITA")

Output: IIIIIIIIIIIIII REJEITA
Success!


Além disso, outros cenários foram explorados, os quais foram testados a seguir:

In [196]:
expect(tm.compute("IIII#II"), "IIII#II= ACEITA")

Output: IIII#II= ACEITA
Success!


In [197]:
expect(tm.compute("IIII#"), "IIII# REJEITA")

Output: IIII# REJEITA
Success!


In [198]:
expect(tm.compute("#"), "# REJEITA")

Output: # REJEITA
Success!


In [199]:
expect(tm.compute("#I"), "#I REJEITA")

Output: #I REJEITA
Success!
